In [1]:
import numpy as np
import pandas as pd
import sys
from datetime import datetime
import os
from IPython.display import display, SVG
import json

sys.path.insert(1, '/global/homes/b/bkieft/metatlas')
import notebooks.standards_library.standard_annotation as sta
pd.options.display.max_colwidth = 300

timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

# Enter Reference Standard information

In [2]:
# Load configuration
config_path = "/global/homes/b/bkieft/metatlas/notebooks/standards_library/msms_set_config.json"
with open(config_path, "r") as config_file:
    config = json.load(config_file)

# Get EIC and Spectra information from run table

In [3]:
if config["new_full_data"] is True:
    lcmsruns_table_with_adducts = sta.build_standard_lcmsrun_table(config['standards_input_file'], include_polarities=config["include_polarities"], 
                                                                   include_chromatographies=config["include_chromatographies"], include_adducts=config["include_adducts"])

    eics_full, top_spectra_full, group_names_full, rt_peaks_full, atlas_full, mols_images = sta.extract_data(lcmsruns_table=lcmsruns_table_with_adducts,
                                                                                                            ppm_tolerance=config["ppm_tolerance"],
                                                                                                            method="find_peaks")

# Save and/or read full data

In [4]:
if config["new_full_data"] is True:
    sta.handle_data(
        mode="save",
        save_path=config['standards_output_path'],
        timestamp=timestamp,
        data=(eics_full, top_spectra_full, group_names_full, rt_peaks_full, atlas_full, mols_images),
        file_suffix="full"
    )

elif config["new_full_data"] is False:
    eics_full, top_spectra_full, group_names_full, rt_peaks_full, atlas_full, mols_images = sta.handle_data(mode="load",
                                                                                                            save_path=config['standards_output_path'],
                                                                                                            file_suffix="full")

Loading most recent pkl file: /global/homes/b/bkieft/metabolomics_standards_annotation/standard_library_annotation/schellermetasci/cache/20250429142914_ref_stds_full.pkl


# Create interactive plot and choose adducts

In [5]:
if config["new_selected_data"] is True:

    processed_data = sta.process_data_for_plotting(eics_full, top_spectra_full, group_names_full, rt_peaks_full, config["include_adducts"], sort_by='run', # 'run', 'specs'
                                                   subset_by_compound="sorgoleone", subset_by_run=None) # Use these to subset by compound (str) or run number (int)

    selected_adducts_dict = {}
    ambiguous_adducts_dict = {}
    top_adducts_dict = {}
    sta.create_interactive_plots(processed_data, mols_images, selected_adducts_dict, ambiguous_adducts_dict, top_adducts_dict)

Output()

# Save and/or read selected data

In [6]:
if config["new_selected_data"] is True:
    sta.handle_data(
        mode="save",
        save_path=config['standards_output_path'],
        timestamp=timestamp,
        data=(selected_adducts_dict, ambiguous_adducts_dict, top_adducts_dict, processed_data),
        file_suffix="selected"
    )
    
elif config["new_selected_data"] is False:
    selected_adducts_dict, ambiguous_adducts_dict, top_adducts_dict, processed_data = sta.handle_data(mode="load",
                                                                                                        save_path=config['standards_output_path'],
                                                                                                        file_suffix="selected")

Saving data to: /global/homes/b/bkieft/metabolomics_standards_annotation/standard_library_annotation/schellermetasci/cache/20250429151023_ref_stds_selected.pkl


# Save summary plots of selected compounds+adducts after completing GUI

In [7]:
if config["new_selected_data"] is True:
    sta.generate_static_summary_plots(processed_data, selected_adducts_dict, top_adducts_dict, config["standards_output_path"])

 Processing selected compounds:   0%|          | 0/8 [00:00<?, ?compound group/s]

[15:11:24] non-ring atom 0 marked aromatic
[15:11:24] non-ring atom 0 marked aromatic


# Filter RT Peak, EICs, and Top Spectra by selected compounds+adducts

In [8]:
if config["new_filtered_data"] is True:
    eics_filtered, rt_peaks_filtered, top_spectra_filtered = sta.filter_by_selected(eics_full, rt_peaks_full, top_spectra_full, 
                                                                                    selected_adducts_dict, top_adducts_dict)


Total unique compounds retained: 1
Total unique compound+adduct entries retained: 10

Total EICs selected: 31
Total RT peaks selected: 12
Total MS2 spectra selected: 12


# Save and/or read filtered data

In [9]:
if config["new_filtered_data"] is True:
    sta.handle_data(
        mode="save",
        save_path=config['standards_output_path'],
        timestamp=timestamp,
        data=(eics_filtered, top_spectra_filtered, rt_peaks_filtered),
        file_suffix="filtered"
    )
    
elif config["new_filtered_data"] is False:
    eics_filtered, top_spectra_filtered, rt_peaks_filtered = sta.handle_data(mode="load",
                                                                             save_path=config['standards_output_path'],
                                                                             file_suffix="filtered")

Saving data to: /global/homes/b/bkieft/metabolomics_standards_annotation/standard_library_annotation/schellermetasci/cache/20250429151023_ref_stds_filtered.pkl


# Format RT peaks data for adding to metatlas DB, atlases, and MSMS refs

In [10]:
rt_peaks_filtered_all_selected, rt_peaks_filtered_top_selected = sta.format_and_select_top_adducts(rt_peaks_filtered, top_adducts_dict)

Working on dataset: top
	Checking for differing RTs between CEs and polarities, which are unexpected...
		Group ('C18', 'sorgoleone'): All RT values for ['102040norm' '205060norm'] and ['POS' 'NEG'] are within 0.05 mins of each other (0.0057).
		Group ('HILICZ', 'sorgoleone'): All RT values for ['102040norm' '205060norm'] and ['POS' 'NEG'] are within 0.05 mins of each other (0.0107).

	Grouping by monoisotopic_mass and identify isomers in the datasets...
		No isomers found in unfiltered data.

	Selecting best collision energy row by intensity for the top adduct(s) per compound...

Working on dataset: all
	Checking for differing RTs between CEs and polarities, which are unexpected...
		Group ('C18', 'sorgoleone'): All RT values for ['102040norm' '205060norm'] and ['NEG' 'POS'] are within 0.05 mins of each other (0.0057).
		Group ('HILICZ', 'sorgoleone'): All RT values for ['102040norm' '205060norm'] and ['NEG' 'POS'] are within 0.05 mins of each other (0.0107).

	Grouping by monoisotopi

# Check if selected compounds are in metatlas DB

In [11]:
in_db, notin_db = sta.search_for_matches_in_metatlas_db(rt_peaks_filtered_top_selected, check_by_flat=True)

 Searching for matches in metatlas db:   0%|          | 0/5 [00:00<?, ?compound/s]


Summary of compounds already in the metatlas database:



,query_label,query_matching_criterion,query_to_db,db_match
0,sorgoleone,inchi_key,FGWRUVXUQWGLOX-AFJQJTPPSA-N,[FGWRUVXUQWGLOX-AFJQJTPPSA-N]



All compounds are already in the metatlas database.



# Store selected compounds+adducts in metatlas db

In [ ]:
# if len(notin_db) > 0:
#     sta.store_in_metatlas_db(notin_db)

In [ ]:
# sta.test_metatlas_db_insertion(inchi_key="FGWRUVXUQWGLOX-AFJQJTPPSA-N", table="Compounds")

# Check if selected compounds/adducts are in the atlases

In [16]:
ema_atlases_data = sta.get_ema_atlas_data(config["current_ema_atlases"])

In [17]:
rt_peaks_filtered_top_selected_formatted = sta.convert_rt_peaks_to_atlas_format(rt_peaks_filtered_top_selected)

In [18]:
matches_to_atlases, nonmatches_to_atlases = sta.search_for_matches_in_atlases(rt_peaks_filtered_top_selected_formatted, ema_atlases_data)

 Searching for matches in existing atlases:   0%|          | 0/5 [00:00<?, ?compound/s]


None of these compounds+adducts were found in the atlases.


These compounds+adducts are not yet in any atlases:



,standard_lcmsrun,chromatography,compound_name,adduct,polarity,rt_peak,mz,smiles,peak_index,inchi,...,permanent_charge,formula,mono_isotopic_molecular_weight,collision_energy,label,rt_min,rt_max,mz_tolerance,mz_tolerance_units,in_metatlas
0,/global/cfs/cdirs/metatlas/raw_data/jgi/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782_POS_MS2_RefStd-1_Sorgoleone_1_Rg80to1200-CE102040norm-200uM-S1_Run382.h5,C18,sorgoleone,[M+H]+,positive,7.543334,359.221685,COC1=CC(=O)C(=C(C1=O)CCCCCCC/C=C\C/C=C\CC=C)O,peak1,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",...,0,C22H30O4,358.214409,102040norm,sorgoleone,7.043334,8.043334,5,ppm,True
1,/global/cfs/cdirs/metatlas/raw_data/jgi/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782_POS_MS2_RefStd-1_Sorgoleone_1_Rg80to1200-CE205060norm-200uM-S1_Run383.h5,C18,sorgoleone,[M+Na]+,positive,7.544277,381.203627,COC1=CC(=O)C(=C(C1=O)CCCCCCC/C=C\C/C=C\CC=C)O,peak1,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",...,0,C22H30O4,358.214409,205060norm,sorgoleone,7.044277,8.044277,5,ppm,True
2,/global/cfs/cdirs/metatlas/raw_data/jgi/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782_NEG_MS2_RefStd-1_Sorgoleone_1_Rg80to1200-CE205060norm-200uM-S1_Run385.h5,C18,sorgoleone,[M-H]-,negative,7.547524,357.207133,COC1=CC(=O)C(=C(C1=O)CCCCCCC/C=C\C/C=C\CC=C)O,peak1,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",...,0,C22H30O4,358.214409,205060norm,sorgoleone,7.047524,8.047524,5,ppm,True
3,/global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558_POS_MS2_RefStd-1_Sorgoleone_1_Rg70to1050-CE102040norm-200uM-S1_Run375.h5,HILICZ,sorgoleone,[M+H]+,positive,0.801184,359.221685,COC1=CC(=O)C(=C(C1=O)CCCCCCC/C=C\C/C=C\CC=C)O,peak1,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",...,0,C22H30O4,358.214409,102040norm,sorgoleone,0.301184,1.301184,5,ppm,True
4,/global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558_NEG_MS2_RefStd-1_Sorgoleone_1_Rg70to1050-CE102040norm-200uM-S1_Run377.h5,HILICZ,sorgoleone,[M-H]-,negative,0.804263,357.207133,COC1=CC(=O)C(=C(C1=O)CCCCCCC/C=C\C/C=C\CC=C)O,peak1,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",...,0,C22H30O4,358.214409,102040norm,sorgoleone,0.304263,1.304263,5,ppm,True


In [20]:
if config["new_rt_correction_data"] is True:
    baseline_to_experimental_qc = sta.get_qc_experimental_atlas(nonmatches_to_atlases, config["current_qc_atlases"], include_istds=True)

Getting raw QC files for /global/cfs/cdirs/metatlas/raw_data/jgi/20241021_JGI_HS_510060_SorghExu_final1_IQX_C18_USDAY92782...

Retrieving baseline C18 QC atlas...




Getting raw QC files for /global/cfs/cdirs/metatlas/raw_data/jgi/20241002_JGI_HS_510060_SorghExu_final1_EXP120B_HILICZ_USHXG02558...

Retrieving baseline HILICZ QC atlas...




In [21]:
if config["new_rt_correction_data"] is True:
    sta.handle_data(
        mode="save",
        save_path=config['standards_output_path'],
        timestamp=timestamp,
        data=(baseline_to_experimental_qc),
        file_suffix="rt_correction"
    )
    
elif config["new_rt_correction_data"] is False:
    baseline_to_experimental_qc = sta.handle_data(mode="load",
                                                  save_path=config['standards_output_path'],
                                                  file_suffix="rt_correction")

Saving data to: /global/homes/b/bkieft/metabolomics_standards_annotation/standard_library_annotation/schellermetasci/cache/20250429151023_ref_stds_rt_correction.pkl


In [22]:
baseline_correction_inputs = sta.create_baseline_correction_input(nonmatches_to_atlases, baseline_to_experimental_qc)
baseline_correction_outputs = sta.rt_correction_from_baseline(baseline_correction_inputs, config["include_chromatographies"])

Backward RT correction data for C18:


,label,polarity,rt_peak_baseline,rt_peak_experimental,rt_peak_corrected,rt_min_corrected,rt_max_corrected,rt_diff_experimental_vs_corrected
0,sorgoleone,positive,NaN,7.543334,7.438575,6.938575,7.938575,0.104759
1,sorgoleone,positive,NaN,7.544277,7.439548,6.939548,7.939548,0.104729
2,sorgoleone,negative,NaN,7.547524,7.442898,6.942898,7.942898,0.104627
3,9-cis-retinoic acid,QC,7.372778,7.322109,7.210608,6.710608,7.710608,0.111501
4,ABMBA (unlabeled),QC,4.700000,4.892107,4.738610,4.238610,5.238610,0.153496
5,caffeine,QC,2.652537,2.833037,2.689957,2.189957,3.189957,0.14308
6,enoxolone,QC,6.605928,6.792058,6.666384,6.166384,7.166384,0.125674
7,inosine (U - 15N),QC,1.020000,1.13626,1.033480,0.533480,1.533480,0.10278
8,nigericin,QC,8.559813,8.749709,8.690287,8.190287,9.190287,0.059422
9,"phenylalanine (U - 13C, 15N)",QC,1.452954,1.481643,1.368337,0.868337,1.868337,0.113306


Backward RT correction data for HILICZ:


,label,polarity,rt_peak_baseline,rt_peak_experimental,rt_peak_corrected,rt_min_corrected,rt_max_corrected,rt_diff_experimental_vs_corrected
0,sorgoleone,positive,NaN,0.801184,0.618179,0.118179,1.118179,0.183005
1,sorgoleone,negative,NaN,0.804263,0.621525,0.121525,1.121525,0.182738
2,ABMBA (unlabeled),QC,1.093806,1.21834,1.070658,0.570658,1.570658,0.147682
3,N-acetyl-glucosamine (U - 13C),QC,6.707815,6.418233,6.572290,6.072290,7.072290,-0.154056
4,adenine (U - 15N),QC,2.557602,2.834223,2.807783,2.307783,3.307783,0.02644
5,"alanine (U - 13C, 15N)",QC,13.405091,13.240681,13.401497,12.901497,13.901497,-0.160816
6,"arginine (U - 13C, 15N)",QC,16.939915,17.013107,16.988042,16.488042,17.488042,0.025065
7,"asparagine (U - 13C, 15N)",QC,14.368089,14.252848,14.377046,13.877046,14.877046,-0.124198
8,"aspartic acid (U - 13C, 15N)",QC,16.130360,16.043652,16.079247,15.579247,16.579247,-0.035595
9,"cystine (U - 13C, 15N)",QC,16.904308,16.930342,16.910804,16.410804,17.410804,0.019538


In [23]:
nonmatches_to_atlases_rt_corrected = sta.substitute_corrected_rt_values(nonmatches_to_atlases, baseline_correction_outputs)

Formatted 7 RT-corrected compounds to be inserted into atlases.
Formatted 7 RT-corrected compounds to be inserted into atlases.


# Store selected compound/adduct in metatlas atlases

In [24]:
sta.update_and_save_atlases(ema_atlases_data, nonmatches_to_atlases_rt_corrected, timestamp, config["standards_output_path"], save_atlas=True)


Old atlas: HILIC_EMA-standards_positive.tsv. Shape (373, 65).
New atlas: HILIC_EMA-standards_positive_20250429151023.tsv. Shape (374, 65).

Saving new EMA atlas to disk at /global/homes/b/bkieft/metabolomics_standards_annotation/standard_library_annotation/schellermetasci/updated_EMA_atlases/HILIC_EMA-standards_positive_20250429151023.tsv

Old atlas: HILIC_EMA-standards_negative.tsv. Shape (418, 65).
New atlas: HILIC_EMA-standards_negative_20250429151023.tsv. Shape (419, 65).

Saving new EMA atlas to disk at /global/homes/b/bkieft/metabolomics_standards_annotation/standard_library_annotation/schellermetasci/updated_EMA_atlases/HILIC_EMA-standards_negative_20250429151023.tsv

Old atlas: C18_EMA-standards_positive.tsv. Shape (3792, 65).
New atlas: C18_EMA-standards_positive_20250429151023.tsv. Shape (3796, 65).

Saving new EMA atlas to disk at /global/homes/b/bkieft/metabolomics_standards_annotation/standard_library_annotation/schellermetasci/updated_EMA_atlases/C18_EMA-standards_positi

# Check if selected compounds/adducts are in MSMS refs

In [25]:
msms_refs = sta.get_msms_refs(msms_refs_path=config["current_msms_refs_path"])
msms_refs.shape

(216409, 17)

In [26]:
rt_peaks_filtered_all_selected_with_spectra = sta.merge_selected_peaks_with_top_spectra(rt_peaks_filtered_all_selected, top_spectra_filtered)
rt_peaks_filtered_all_selected_with_spectra_formatted = sta.format_for_msms_refs(rt_peaks_filtered_all_selected_with_spectra, msms_refs, config["msms_refs_metadata"])

In [27]:
in_msms_refs, notin_msms_refs = sta.search_for_matches_in_msms_refs(rt_peaks_filtered_all_selected_with_spectra_formatted, msms_refs, check_by_flat=True)

 Searching for matches in MSMS refs:   0%|          | 0/12 [00:00<?, ?compound/s]


These compounds+adducts are not yet in MSMS refs:



,database,id,name,spectrum,decimal,precursor_mz,polarity,adduct,fragmentation_method,collision_energy,instrument,instrument_type,formula,exact_mass,inchi_key,inchi,smiles
0,metatlas,schellermetasci5c00f62995f646968d6cecb658f6dd06,sorgoleone,"[[55.05423, 57.07008, 67.05420, 69.06986, 79.05420, 81.06986, 82.07323, 83.08547, 91.05445, 93.06970, 95.04946, 95.08557, 96.08879, 97.10124, 107.08566, 108.08907, 108.48547, 109.09235, 109.10123, 110.10484, 117.07016, 119.08595, 121.10121, 122.10472, 123.04414, 123.10750, 123.11723, 135.11691, ...",4.0,359.2217,positive,[M+H]+,HCD,ramp-102040norm,EXP120B,Orbitrap,C22H30O4,358.214409,FGWRUVXUQWGLOX-AFJQJTPPSA-N,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",C=CC/C=C\C/C=C\CCCCCCCC1=C(O)C(=O)C=C(OC)C1=O
1,metatlas,schellermetasci5794675116cd483fa800577c669a9b5b,sorgoleone,"[[57.07003, 59.59517, 62.04410, 66.05342, 67.05431, 69.06971, 71.08553, 81.06994, 85.06497, 85.10103, 89.05969, 95.08563, 109.10144, 123.04451, 123.11723, 133.08659, 135.11713, 139.03918, 153.05482, 163.03949, 165.49907, 169.04941, 170.05376, 183.06538, 193.08685, 195.12248, 205.08527, 219.10349...",4.0,376.2482,positive,[M+NH4]+,HCD,ramp-102040norm,EXP120B,Orbitrap,C22H30O4,358.214409,FGWRUVXUQWGLOX-AFJQJTPPSA-N,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",C=CC/C=C\C/C=C\CCCCCCCC1=C(O)C(=O)C=C(OC)C1=O
2,metatlas,schellermetascid42b8babc6d24f08aad1cb8d9c8ed5dc,sorgoleone,"[[50.59600, 53.03876, 55.05416, 57.06996, 65.03865, 67.05427, 67.05890, 68.05764, 68.99744, 69.07001, 69.15972, 79.05415, 80.05785, 81.06987, 81.07558, 82.07326, 83.08556, 89.22113, 91.05418, 92.05798, 93.03343, 93.06982, 94.07337, 95.04892, 95.08545, 96.08906, 97.10086, 105.06997, 107.04953, 10...",4.0,359.2217,positive,[M+H]+,HCD,ramp-205060norm,EXP120B,Orbitrap,C22H30O4,358.214409,FGWRUVXUQWGLOX-AFJQJTPPSA-N,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",C=CC/C=C\C/C=C\CCCCCCCC1=C(O)C(=O)C=C(OC)C1=O
3,metatlas,schellermetascic858282a453247e89d24d9e6827f8391,sorgoleone,"[[55.05426, 56.76063, 57.03352, 57.06989, 59.15232, 67.05408, 69.06985, 71.08537, 79.05448, 81.06998, 85.06484, 89.06007, 93.07011, 95.04906, 95.08515, 96.08925, 100.48362, 105.06947, 107.08559, 109.10104, 113.09632, 123.04424, 131.08481, 139.03961, 141.05486, 149.02393, 153.05507, 155.03473, 16...",4.0,376.2482,positive,[M+NH4]+,HCD,ramp-205060norm,EXP120B,Orbitrap,C22H30O4,358.214409,FGWRUVXUQWGLOX-AFJQJTPPSA-N,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",C=CC/C=C\C/C=C\CCCCCCCC1=C(O)C(=O)C=C(OC)C1=O
4,metatlas,schellermetasci899f3b6fc74f4ba699893d6b6d82f8ab,sorgoleone,"[[53.68539, 61.73775, 89.80753, 95.01392, 98.00093, 109.02959, 110.00076, 111.00903, 123.00887, 124.01192, 124.04618, 125.01357, 127.00386, 137.02443, 138.02893, 143.79137, 151.00398, 151.04002, 165.01944, 166.02321, 167.02451, 179.03743, 179.07217, 193.08727, 194.09264, 221.08253, 222.08977, 22...",4.0,357.2071,negative,[M-H]-,HCD,ramp-102040norm,EXP120B,Orbitrap,C22H30O4,358.214409,FGWRUVXUQWGLOX-AFJQJTPPSA-N,"InChI=1S/C22H30O4/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-18-21(24)19(23)17-20(26-2)22(18)25/h3,5-6,8-9,17,24H,1,4,7,10-16H2,2H3/b6-5-,9-8-",C=CC/C=C\C/C=C\CCCCCCCC1=C(O)C(=O)C=C(OC)C1=O
5,metatlas,schellermetasci90a471e19c634b22bd55d6aad4d0a341,sorgoleone,"[[52.63279, 68.99819, 73.94276, 74.98546, 92.31145, 95.01382, 96.01723, 98.00102, 106.83733, 109.02973, 110.00115, 111.00905, 123.00885, 124.01189, 125.01354, 125.02384, 127.00364, 128.01149, 137.02452, 137.03735, 138.02769, 139.03047, 151.00417, 151.04027, 152.01048, 165.01941, 165.05583, 165.0...",4.0,357.2071,negative,[M-H]-,HCD,ramp-205060norm,EXP120B,Orbitrap,C22H30O4,358.214409,FGWRUVX

# Store selected compound/adduct in MSMS refs

In [28]:
sta.update_and_save_msms_refs(msms_refs, notin_msms_refs, config["standards_output_path"], timestamp, save_refs=True)

Existing MSMS refs: (216409, 17)
New MSMS refs: (216421, 17)

Saving new MSMS refs to disk at /global/homes/b/bkieft/metabolomics_standards_annotation/standard_library_annotation/schellermetasci/updated_MSMS_refs/msms_refs_20250429151023.tab
